<a href="https://colab.research.google.com/github/ignaciomontovio/TP1-PrograConc/blob/c%2B%2B/TP_1_Parte_1_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir src
!mkdir bin

In [ ]:
%%writefile src/process_tree.cpp
#include <cstring>
#include <iostream>
#include <map>
#include <sstream>
#include <sys/wait.h>
#include <unistd.h>
#include <vector>

using namespace std;

struct Node
{
  char letter;
  pid_t pid;
  pid_t parent;

  Node(char letter_)
  {
    letter = letter_;
    pid = getpid();
    parent = getppid();
  }

  std::string toString()
  {
    std::stringstream aux;
    aux << "Process: " << letter << ", PID: " << pid << ", Parent PID: "
    << parent;
    return aux.str();
  }
};
typedef struct Node Node;

int main(int argc, char *argv[])
{
  map<char, vector<char>> nodes_map =
  {
    {'A', {'B', 'C'}},
    {'B', {'D', 'E'}},
    {'C', {'F'}},
    {'E', {'G', 'H'}},
  };

  Node current = Node('A');
  cout << current.toString() << endl;

  vector<char>::iterator next_child = nodes_map[current.letter].begin();
  vector<char>::iterator end = nodes_map[current.letter].end();

  while (next_child != end)
  {
    pid_t new_child = fork();

    if (new_child < 0)
    {
      cerr << "Error: " << strerror(errno) << endl;
      return EXIT_FAILURE;
    }

    if (new_child == 0)
    {
      current = Node(*next_child);
      cout << current.toString() << endl;

      // Si el hijo tiene que crear mas nodos reiniciamos el iterador.
      if (nodes_map.find(current.letter) != nodes_map.end())
      {
        next_child = nodes_map[current.letter].begin();
        end = nodes_map[current.letter].end();
        continue;
      }

      break; // Sino terminamos el loop.
    }

    next_child++;
  }

  if (nodes_map.find(current.letter) != nodes_map.end())
  {
    next_child = nodes_map[current.letter].begin();
    for (; next_child != end; next_child++)
    {
      wait(NULL);
    }
  }
  else
  {
    unsigned int time = 10;

    if (argc > 1)
    {
      try
      {
        time = stoi(argv[1]);
      }
      catch(const invalid_argument& e)
      {
        cerr << "Invalid time passed as parameter, using default time ("
        << time << " seconds)." << endl;
      }
    }

    sleep(time);
  }

  cout << "Cerrando proceso: " << current.letter << endl;

  return EXIT_SUCCESS;
}

In [ ]:
%%writefile run.sh
#!/usr/bin/env bash
SLEEP_TIME=5

echo "Compilando..."
g++ -std=c++17 -g src/process_tree.cpp -o bin/process_tree

echo "Ejecutando..."
bin/process_tree $SLEEP_TIME 1>output 2>/dev/null &
FIRST_PROCESS=$!

sleep 1 # Esperamos un segundo a que se creen los nodos...
echo "Arbol de procesos:"
pstree -pc $FIRST_PROCESS

sleep $(expr $SLEEP_TIME + 1) # Esperamos a que se cierren...

echo -e "\nSalida del programa:"
cat ./output

In [ ]:
!chmod +x ./run.sh

In [ ]:
!./run.sh